# LIBRARIES

In [1]:
from mltools.Input import input
path="/home/david/Desktop/Project/Bathing water quality 2017 - 2018-05-23"
inp=input.input(XLSX=True)
import os
import pandas as pd
import numpy as np
import re
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, matthews_corrcoef
from sklearn.preprocessing import LabelEncoder
from bokeh.plotting import figure
from bokeh.io import show, output_notebook



warnings.filterwarnings("ignore", category=FutureWarning)

/home/david/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# READ ALL .XLSX FILES AND CREATE A DATAFRAME

In [2]:
folder_list = os.listdir(path)          
df=inp.read(folder_list[0],sheet_name=folder_list[0][:-4])

for file in folder_list[1:]:  
    try:
        a=inp.read(file,sheet_name=file[:-4])
        df=pd.concat([df,a])
        print(len(df))
    except:
        pass
    
df=df.drop_duplicates().reset_index(drop=True)  #remove duplicates and reset_index

Read xlsx file
Impossibile leggere files in map_Italy2.html
finish
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Impossibile leggere files in LV_BW2017.xls
finish
0
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
finish
414
Read xlsx file
Impossibile leggere files in map2.html
finish
414
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
finish
2695
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
finish
3358
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
finish
3396
Read xlsx file
Impossibile leggere files in mltools
finish
3396
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
Impossibile leggere files in PL_BW2017.xls
finish
3396
Read xlsx file
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
finish
3447
Read xlsx file
WARNING

# LOOK VARIABLES

In [3]:
cols=df.columns
cols

Index(['cc', 'BWID', 'SeaRegion', 'Region', 'Province', 'Commune', 'BWName',
       'BWaterCat', 'Longitude_BW', 'Latitude_BW', 'y1990', 'y1991', 'y1992',
       'y1993', 'y1994', 'y1995', 'y1996', 'y1997', 'y1998', 'y1999', 'y2000',
       'y2001', 'y2002', 'y2003', 'y2004', 'y2005', 'y2006', 'y2007', 'y2008',
       'y2009', 'y2010', 'y2011', 'y2012', 'y2013', 'y2014', 'y2015', 'y2016',
       'y2017', 'y2017 comment'],
      dtype='object')

# VARIABLES

In [4]:
for col in cols:
    print(col)
    print("--------------")
    print(df[col].unique())
    print("--------------")
    print("--------------")
    print("--------------")

cc
--------------
['FI' 'GR' 'PT' 'SK' 'RO' 'CH' 'HR' 'AT' 'IT' 'CZ' 'LT' 'SI' 'EE' 'DE'
 'UK' 'LU' 'DK' 'FR' 'SE' 'CY' 'BE' 'BG' 'IE' 'NL' 'HU' 'ES' 'MT' 'AL']
--------------
--------------
--------------
BWID
--------------
['FI181092003 ' 'FI193992006 ' 'FI194145001 ' ... 'AL_807 ' 'AL_808 '
 'AL_809 ']
--------------
--------------
--------------
SeaRegion
--------------
['Baltic Sea' nan 'North-east Atlantic Ocean' 'Mediterranean Sea'
 'Black Sea' 'North Sea' ' ' 'Northwest Atlantic Ocean' 'Indian Ocean']
--------------
--------------
--------------
Region
--------------
['ETELAE-SUOMI' nan 'LAENSI-SUOMI' 'POHJOIS-SUOMI' 'ITAE-SUOMI'
 'AHVENANMAA' 'PELOPONNISOS' 'KENTRIKI MAKEDONIA' 'KRITI' 'VOREIO AIGAIO'
 'ANATOLIKI MAKEDONIA, THRAKI' 'DYTIKI MAKEDONIA' 'THESSALIA' 'IPEIROS'
 'IONIA NISIA' 'DYTIKI ELLADA' 'STEREA ELLADA' 'ATTIKI' 'NOTIO AIGAIO'
 'ACORES' 'CENTRO' 'LISBOA E VALE DO TEJO' 'NORTE' 'ALGARVE' 'ALENTEJO'
 'MADEIRA' 'SLOVENSKA REPUBLIKA' 'RO MACRO2 SE' 'SOUTH-EAST' 'SW

# A LITTLE LOOK THE VARIABLES

#### COUNT BATHING WATER FOR EACH COUNTRY

In [5]:
DIC={}
for k in df['cc'].unique():
    
    DIC[k]=len(df[df['cc']==k])
    
DIC

{'FI': 414,
 'GR': 2281,
 'PT': 663,
 'SK': 38,
 'RO': 51,
 'CH': 504,
 'HR': 1045,
 'AT': 275,
 'IT': 6190,
 'CZ': 202,
 'LT': 129,
 'SI': 48,
 'EE': 76,
 'DE': 2449,
 'UK': 651,
 'LU': 23,
 'DK': 1355,
 'FR': 3744,
 'SE': 885,
 'CY': 118,
 'BE': 141,
 'BG': 96,
 'IE': 144,
 'NL': 801,
 'HU': 314,
 'ES': 2464,
 'MT': 87,
 'AL': 102}

# ABOUT BWID

#### PROBABLY IT IS A VARIABLE "key" SO IT IS USELESS FOR THIS ANALYSIS

In [6]:
print(len(df.BWID.unique())==len(df)) #is unique

df=df.drop('BWID',axis=1)

True


# ABOUT BWATERCAT

#### THERE ARE SOME RARE ELEMENT, SO I DROP THEM. ALSO, WE CAN SEE THERE IS SOME ROWS WITH 
#### " R" INSTEAD THAN "R", SO I WILL CORRECT IT

In [7]:
print(df['BWaterCat'].value_counts())
df['BWaterCat'][df['BWaterCat']=='R ']='R'
df = df.drop(df[df.BWaterCat.isin(['EM','estuary'])].index) #remove few frequency values
df['BWaterCat'][df['BWaterCat']=='R']='River'
df['BWaterCat'][df['BWaterCat']=='C']='Coast'
df['BWaterCat'][df['BWaterCat']=='L']='Lake'
df['BWaterCat'][df['BWaterCat']=='T']='Transitional'
df.BWaterCat.unique()



C          16640
L           7045
R           1313
T            273
estuary       14
R              2
EM             1
Name: BWaterCat, dtype: int64


/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarnin

array(['Lake', 'Coast', 'River', 'Transitional', nan], dtype=object)

# ABOUT NUMERIC VARIABLES

#### LATITUDINE AND LONGITUDE I THINK ARE NICE VARIABLES FOR PREDICTING IMPORT GEO-INFORMATION (REGION, PROVINCE, COMMUNE, SEAREGION), SO I DROPPED THE ROWS WITH NaN AND THE POSSIBLE OUTLIERS

In [8]:
print(len(df))
df['Longitude_BW']=df['Longitude_BW'].fillna(181) 
df = df.drop(df[(df.Longitude_BW>180) | (df.Longitude_BW< -180) ].index)
df['Latitude_BW']=df['Latitude_BW'].fillna(91)
df = df.drop(df[(df.Latitude_BW>90) | (df.Latitude_BW< -90)].index)

df=df.reset_index(drop=True)

df
print(len(df))


25275
23594


# ABOUT QUALITY

#### "QUALITY" IS VERY IMPORTANT VARIABLE, BUT THE PROBLEM IS THAT IT IS A QUALITIVE VARIABLE, BUT FOR CONFRONTING WE NEED A QUANTITIVE VARIABLE. SO I CREATED IT WITH THE FREQUENCY FOR ALL THE DIFFERENT CASES OF THIS VARIABLE. I CONSIDERED JUST COUNTRIES OF THE MEDITERRANEA AREA (WHERE THERE IS THE ITALY TOO).

In [9]:
df_filt=df[df['SeaRegion']=='Mediterranean Sea']


df_filt.head(10)

,cc,SeaRegion,Region,Province,Commune,BWName,BWaterCat,Longitude_BW,Latitude_BW,y1990,...,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017,y2017 comment
454,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,4KM NATIONAL ROAD ALEXANDROUPOLI-KOMOTINI,Coast,25.8145,40.8509,NaN,...,NF,CG,excellent,excellent,excellent,NaN,NaN,NaN,NaN,NaN
455,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,KALAMAKI,Coast,25.8269,40.8490,NF,...,NF,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
456,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,DELPHINI,Coast,25.8395,40.8479,NaN,...,NF,CI,excellent,good,good,excellent,excellent,excellent,excellent,NaN
457,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,DIMOTIKH PLAZ ALEXANDROUPOLIS,Coast,25.8560,40.8448,NF,...,CG,CI,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
458,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,"AG,PARASKEVI",Coast,25.7287,40.8464,NF,...,NF,CG,excellent,excellent,excellent,NaN,NaN,NaN,NaN,NaN
459,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,AGIOS GEORGIOS,Coast,25.7571,40.8484,NF,...,NF,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
460,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,APALOS - PLAZ DIMOU,Coast,25.9364,40.8439,NaN,...,NF,CG,excellent,excellent,excellent,NaN,NaN,NaN,NaN,NaN
461,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,DIKELLA,Coast,25.6747,40.8549,NF,...,NF,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
462,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,ALEXANDROUPOLI,KYANI AKTI,Coast,25.7245,40.8472,NaN,...,CG,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
463,GR,Mediterranean Sea,"ANATOLIKI MAKEDONIA, THRAKI",EVROS,SAMOTHRAKI,FONIAS,Coast,25.6577,40.4931,NaN,...,NF,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN


#### PLOT OF THE DIFFERENT LEVELS OF QUALITY 

In [10]:
f=df_filt.y2013[df_filt.y2013.isin(['excellent','good','poor','sufficient'])]


f[f=='excellent']='10.1'
f[f=='good']='7'
f[f=='sufficient']='4'
f[f=='poor']='1'
f=f.astype(float)


f2=df_filt.y2014[df_filt.y2014.isin(['excellent','good','poor','sufficient'])]


f2[f2=='excellent']='10.15'
f2[f2=='good']='7.3'
f2[f2=='sufficient']='4.3'
f2[f2=='poor']='1.3'
f2=f2.astype(float)

f3=df_filt.y2015[df_filt.y2015.isin(['excellent','good','poor','sufficient'])]


f3[f3=='excellent']='10.20'
f3[f3=='good']='7.6'
f3[f3=='sufficient']='4.6'
f3[f3=='poor']='1.6'
f3=f3.astype(float)

f4=df_filt.y2016[df_filt.y2016.isin(['excellent','good','poor','sufficient'])]


f4[f4=='excellent']='10.25'
f4[f4=='good']='7.9'
f4[f4=='sufficient']='4.9'
f4[f4=='poor']='1.9'
f4=f4.astype(float)

f4=df_filt.y2016[df_filt.y2016.isin(['excellent','good','poor','sufficient'])]


f4[f4=='excellent']='10.255'
f4[f4=='good']='8'
f4[f4=='sufficient']='5'
f4[f4=='poor']='2'
f4=f4.astype(float)

f5=df_filt.y2017[df_filt.y2017.isin(['excellent','good','poor','sufficient'])]


f5[f5=='excellent']='10.305'
f5[f5=='good']='8.3'
f5[f5=='sufficient']='5.5'
f5[f5=='poor']='2.3'
f5=f5.astype(float)

print(df)



       cc          SeaRegion         Region      Province   Commune  \
0      FI         Baltic Sea   ETELAE-SUOMI       UUSIMAA       NaN   
1      FI                NaN            NaN           NaN       NaN   
2      FI                NaN   LAENSI-SUOMI           NaN       NaN   
3      FI                NaN            NaN           NaN       NaN   
4      FI                NaN            NaN           NaN       NaN   
5      FI                NaN   ETELAE-SUOMI           NaN       NaN   
6      FI                NaN  POHJOIS-SUOMI           NaN       NaN   
7      FI         Baltic Sea   ETELAE-SUOMI  ITAE-UUSIMAA    ASKOLA   
8      FI         Baltic Sea   ETELAE-SUOMI       UUSIMAA     ESPOO   
9      FI         Baltic Sea   ETELAE-SUOMI       UUSIMAA     ESPOO   
10     FI         Baltic Sea   ETELAE-SUOMI       UUSIMAA     ESPOO   
11     FI         Baltic Sea   ETELAE-SUOMI       UUSIMAA     ESPOO   
12     FI         Baltic Sea   ETELAE-SUOMI       UUSIMAA     ESPOO   
13    

In [11]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label


from numpy import histogram, linspace
from scipy.stats.kde import gaussian_kde

#pdf = gaussian_kde(f)

#x = linspace(0,250,200)

p = figure(plot_width = 1000, plot_height = 600, 
           title = 'QUALITY',
           x_axis_label = 'QUALITY', y_axis_label = '#')
#p.line(x, pdf(x))

# plot actual hist for comparison
hist, edges = histogram(f[f<10], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,legend='2013')
hist, edges = histogram(f2[f2<10], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='red',legend='2014')
hist, edges = histogram(f3[f3<10], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='green',legend='2015')
hist, edges = histogram(f4[f4<10], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='yellow',legend='2016')
hist, edges = histogram(f5[f5<10], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='orange',legend='2017')
mytext = Label(x=1.5, y=200, text='POOR')
mytext2 = Label(x=4.5, y=200, text='SUFFICIENT')
mytext3 = Label(x=7.25, y=620, text='GOOD')

p.add_layout(mytext)
p.add_layout(mytext2)
p.add_layout(mytext3)


output_notebook()

show(p)




p = figure(plot_width = 1000, plot_height = 400, 
           title = 'QUALITY',
           x_axis_label = 'QUALITY', y_axis_label = '#')
#p.line(x, pdf(x))

# plot actual hist for comparison
hist, edges = histogram(f[f>9], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,legend='2013')
hist, edges = histogram(f2[f2>9], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='red',legend='2014')
hist, edges = histogram(f3[f3>9], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='green',legend='2015')
hist, edges = histogram(f4[f4>9], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='yellow',legend='2016')
hist, edges = histogram(f5[f5>9], density=False, bins=20)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], alpha=0.4,color='orange',legend='2017')
mytext = Label(x=10.5, y=6000, text='EXCELLENT')


p.add_layout(mytext)



output_notebook()

show(p)

Loading BokehJS ...

Loading BokehJS ...

CALCULATE VALUE OF EACH RANK OF QUALITY

In [12]:
f=df_filt[df_filt.isin(['excellent','good','poor','sufficient'])]




f=f[['y2013','y2014','y2015','y2016','y2017']]
typ='excellent'
excellent=len(df.y2013[df.y2013==typ])+len(df.y2014[df.y2014==typ])+len(df.y2015[df.y2015==typ])+len(df.y2016[df.y2016==typ])+len(df.y2017[df.y2017==typ])
print(excellent)


typ='good'
good=len(df.y2013[df.y2013==typ])+len(df.y2014[df.y2014==typ])+len(df.y2015[df.y2015==typ])+len(df.y2016[df.y2016==typ])+len(df.y2017[df.y2017==typ])
print(good)

typ='sufficient'
sufficient=len(df.y2013[df.y2013==typ])+len(df.y2014[df.y2014==typ])+len(df.y2015[df.y2015==typ])+len(df.y2016[df.y2016==typ])+len(df.y2017[df.y2017==typ])
print(sufficient)

typ='poor'
poor=len(df.y2013[df.y2013==typ])+len(df.y2014[df.y2014==typ])+len(df.y2015[df.y2015==typ])+len(df.y2016[df.y2016==typ])+len(df.y2017[df.y2017==typ])
print(poor)

#I Imagine that "poor" and "sufficient" are negative values instead "good" and "excellent" are positive values. 
#"excellent" is very common level, so this increase value of 1/f_excellent, where f_excellent is how many
#excellent there are in dataset. Instead "poor" decrese value of 1/f_poor, where in this case f_poor <f_excellent
#so decrease much more than when increase f_excellent

88952
9196
2555
1827


# PREDICT SEAREGION

#### THERE ARE SOME NaN SO I CAN TRY PREDICT THEM WITH SOME SIMPLE MACHINE LEARNING MODEL 

In [13]:
print("#Searegions: "+str(len(df.SeaRegion.unique())))
print("Searegions: "+str(df.SeaRegion.unique()))

#Searegions: 8
Searegions: ['Baltic Sea' nan 'North-east Atlantic Ocean' 'Mediterranean Sea'
 'Black Sea' 'North Sea' 'Northwest Atlantic Ocean' 'Indian Ocean']


#### FILL NaN WITH " " AND REMOVE RARE VALUES

In [14]:
df.SeaRegion=df.SeaRegion.fillna(' ')   #put Na with " "
df.SeaRegion.unique()

array(['Baltic Sea', ' ', 'North-east Atlantic Ocean',
       'Mediterranean Sea', 'Black Sea', 'North Sea',
       'Northwest Atlantic Ocean', 'Indian Ocean'], dtype=object)

In [15]:
print("HOW MANY: "+str(df.SeaRegion.value_counts()))
print("")
print("")

df = df.drop(df[df.SeaRegion=='North Sea'].index)
print("AFTER REMOVE VALUES WITH FEW FREQUENCY: "+str(df.SeaRegion.value_counts()))

HOW MANY: Mediterranean Sea            11508
North-east Atlantic Ocean     7902
Baltic Sea                    1926
Black Sea                     1142
                               874
Northwest Atlantic Ocean       213
Indian Ocean                    27
North Sea                        2
Name: SeaRegion, dtype: int64


AFTER REMOVE VALUES WITH FEW FREQUENCY: Mediterranean Sea            11508
North-east Atlantic Ocean     7902
Baltic Sea                    1926
Black Sea                     1142
                               874
Northwest Atlantic Ocean       213
Indian Ocean                    27
Name: SeaRegion, dtype: int64


In [16]:
filter_df=df[['cc','SeaRegion','Region','Province','Commune','BWaterCat','Longitude_BW','Latitude_BW']].fillna('UNK')
ind_noempty=filter_df[filter_df.SeaRegion!=' '].index.values
ind_empty=filter_df[filter_df.SeaRegion==' '].index.values
ind_empty
#take index of variable than we want to predict

array([    1,     2,     3,     4,     5,     6,   340,   341,   342,
         343,   344,   345,   346,   347,   348,   349,   350,   351,
         352,   353,   354,   355,   356,   357,   358,   359,   360,
         361,   362,   363,   364,   365,   366,   367,   368,   369,
         370,   371,   372,   373,   374,   375,   376,   377,   378,
         379,   380,   381,   382,   383,   384,   385,   386,   387,
         388,   389,   390,   391,   392,   393,   394,   395,   396,
         397,   398,   399,   400,   401,   402,   403,   404,   405,
         406,   407,   408,   409,   410,   411,   412,   413,   414,
         415,   416,   417,   418,   419,   420,   421,   422,   423,
         424,   425,   426,   427,   428,   429,   430,   431,   432,
         433,   434,   435,   436,   437,   438,   439,   440,   441,
         442,   443,   444,   445,   446,   447,   448,   449,   450,
         451,   452,   453,  2621,  2622,  2623,  2624,  2625,  2626,
        2627,  2628,

In [17]:
filter_df=pd.get_dummies(filter_df[['cc','Region','Province','Commune','BWaterCat','Longitude_BW','Latitude_BW']]) 
filter_df
#trasform the qualitive variables in onehotencoder

,Longitude_BW,Latitude_BW,cc_AL,cc_AT,cc_BE,cc_BG,cc_CH,cc_CY,cc_CZ,cc_DE,...,Commune_ZUPANOVICE,Commune_ZUYDCOOTE,Commune_ZUZEMBERK,Commune_ZWETTL,Commune_ZYGI,BWaterCat_Coast,BWaterCat_Lake,BWaterCat_River,BWaterCat_Transitional,BWaterCat_UNK
0,24.8811,60.2700,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,25.7321,62.6064,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,22.4895,62.6935,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,22.9098,62.2782,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,25.3020,60.2875,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,25.6474,61.0012,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,25.6810,65.1039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,25.5987,60.5414,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,24.7730,60.1537,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,24.6257,60.1536,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
X=filter_df[filter_df.index.isin(ind_noempty)]
Y=df['SeaRegion'][df.index.isin(ind_noempty)]

lb = LabelEncoder()
Y = lb.fit_transform(Y)
print(Y)

X_train, X_test, y_train, y_test = train_test_split(X,Y, 
                                                    test_size=0.3, 
                                                    random_state=40)

print("train shape: " +str(X_train.shape)+"  test shape: "+ str(X_test.shape))

clf=RandomForestClassifier(n_estimators=100)
scores = cross_val_score(clf, X_train, y_train, cv=7,scoring='f1_macro')
print("f1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



#apply a Random Forest with cross valutation after that i split the dataset

[0 0 0 ... 3 3 3]
train shape: (15902, 8012)  test shape: (6816, 8012)
f1-score: 0.99 (+/- 0.01)


In [20]:
rf = clf.fit(X_train, y_train) 
y_predicted = rf.predict(X_test)

print("F1 Random Forest on test set:", f1_score(y_test, y_predicted,average='macro'))

#test model

F1 Random Forest on test set: 0.9924549739414683


In [21]:
df['SeaRegion'][df.index.isin(ind_empty)]=lb.inverse_transform(rf.predict(filter_df[filter_df.index.isin(ind_empty)]))
df['SeaRegion'].unique()

#predict the variable

/home/david/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


array(['Baltic Sea', 'North-east Atlantic Ocean', 'Mediterranean Sea',
       'Black Sea', 'Northwest Atlantic Ocean', 'Indian Ocean'],
      dtype=object)

In [22]:
df['SeaRegion']
print(df.SeaRegion.value_counts())

Mediterranean Sea            11946
North-east Atlantic Ocean     8280
Baltic Sea                    1950
Black Sea                     1175
Northwest Atlantic Ocean       214
Indian Ocean                    27
Name: SeaRegion, dtype: int64


# PREDICT REGION

#### THE SAME FOR REGION 

In [23]:
print("#Region: "+str(len(df.Region.unique())))
print("Regions: "+str(df.Region.unique()))

#Region: 206
Regions: ['ETELAE-SUOMI' nan 'LAENSI-SUOMI' 'POHJOIS-SUOMI' 'ITAE-SUOMI'
 'AHVENANMAA' 'PELOPONNISOS' 'KENTRIKI MAKEDONIA' 'KRITI' 'VOREIO AIGAIO'
 'ANATOLIKI MAKEDONIA, THRAKI' 'DYTIKI MAKEDONIA' 'THESSALIA' 'IPEIROS'
 'IONIA NISIA' 'DYTIKI ELLADA' 'STEREA ELLADA' 'ATTIKI' 'NOTIO AIGAIO'
 'ACORES' 'CENTRO' 'LISBOA E VALE DO TEJO' 'NORTE' 'ALGARVE' 'ALENTEJO'
 'MADEIRA' 'SLOVENSKA REPUBLIKA' 'RO MACRO2 SE' 'SOUTH-EAST' 'SWITZERLAND'
 'ZAGREB' 'CONTINENTAL CROATIA' 'DALMATIA' 'KVARNER' 'ISTRIA' 'SLAVONIA'
 'BURGENLAND' 'NIEDEROESTERREICH' 'WIEN' 'KAERNTEN' 'STEIERMARK'
 'OBEROESTERREICH' 'SALZBURG' 'TIROL' 'VORARLBERG' 'LOMBARDIA'
 'PROV. AUT. DI TRENTO' 'EMILIA ROMAGNA' 'TOSCANA' 'PUGLIA' 'SICILIA'
 'PIEMONTE' 'LIGURIA' 'PROV. AUT. DI BOLZANO' 'VENETO'
 'FRIULI VENEZIA GIULIA' 'UMBRIA' 'MARCHE' 'LAZIO' 'ABRUZZO' 'MOLISE'
 'CAMPANIA' 'BASILICATA' 'CALABRIA' 'SARDEGNA' 'VYSOCINA'
 'JIHOMORAVSKY KRAJ' 'STREDOCESKY KRAJ' 'HL. M. PRAHA                  '
 'STREDOCESKY KRAJ     

In [24]:
def remove(sentence):
    try:
        pattern = re.compile(r'\s+')
        sentence = re.sub(pattern, '', sentence)
    except:
        pass
    return sentence

df['Region']=df['Region'].apply(remove)
vc=df.Region.value_counts()

vals_to_remove = vc[vc <= 30].index.values

df = df.drop(df[df.Region.isin(vals_to_remove)].index)

#THERE ARE SOME REGION WITH MULTIPLE SPACE OR OTHERS ERROR, SO I CORRECTED THEM. ALSO, I REMOVED VERY RARE VALUES
#OF THIS VARIABLE

In [135]:
print("#Region: "+str(len(df.Region.unique())))
print("Regions: "+str(df.Region.unique()))

#Region: 119
Regions: ['ETELAE-SUOMI' nan 'LAENSI-SUOMI' 'POHJOIS-SUOMI' 'ITAE-SUOMI'
 'PELOPONNISOS' 'KENTRIKIMAKEDONIA' 'KRITI' 'VOREIOAIGAIO'
 'ANATOLIKIMAKEDONIA,THRAKI' 'THESSALIA' 'IPEIROS' 'IONIANISIA'
 'DYTIKIELLADA' 'STEREAELLADA' 'ATTIKI' 'NOTIOAIGAIO' 'ACORES' 'CENTRO'
 'LISBOAEVALEDOTEJO' 'NORTE' 'ALGARVE' 'MADEIRA' 'SLOVENSKAREPUBLIKA'
 'ROMACRO2SE' 'SWITZERLAND' 'DALMATIA' 'KVARNER' 'ISTRIA'
 'NIEDEROESTERREICH' 'KAERNTEN' 'STEIERMARK' 'OBEROESTERREICH' 'SALZBURG'
 'TIROL' 'LOMBARDIA' 'PROV.AUT.DITRENTO' 'EMILIAROMAGNA' 'TOSCANA'
 'PUGLIA' 'SICILIA' 'PIEMONTE' 'LIGURIA' 'VENETO' 'FRIULIVENEZIAGIULIA'
 'MARCHE' 'LAZIO' 'ABRUZZO' 'MOLISE' 'CAMPANIA' 'BASILICATA' 'CALABRIA'
 'SARDEGNA' 'SLOVENIJA' 'ESTONIA' 'BRANDENBURG' 'BADEN-WUERTTEMBERG'
 'NORDRHEIN-WESTFALEN' 'SACHSEN' 'SACHSEN-ANHALT' 'THUERINGEN' 'BAYERN'
 'NIEDERSACHSEN' 'HESSEN' 'MECKLENBURG-VORPOMMERN' 'RHEINLANDPFALZ'
 'SCHLESWIG-HOLSTEIN' 'NORTHEAST(UK)' 'NORTHWEST(UK)' 'EASTOFENGLAND'
 'SOUTHEAST(UK)' 'SOUTHWEST

In [25]:
df['Region']=df['Region'].fillna("")
len(df['Region'][df['Region']==''])

660

In [26]:
filter_df=df[['cc','SeaRegion','Region','Province','Commune','BWaterCat','Longitude_BW','Latitude_BW']].fillna('UNK')
ind_noempty=filter_df[filter_df.Region!=''].index.values
ind_empty=filter_df[filter_df.Region==''].index.values
print(ind_empty)
filter_df=pd.get_dummies(filter_df[['cc','SeaRegion','Province','Commune','BWaterCat','Longitude_BW','Latitude_BW']]) 


X=filter_df[filter_df.index.isin(ind_noempty)]
Y=df['Region'][df.index.isin(ind_noempty)]

lb = LabelEncoder()
Y = lb.fit_transform(Y)
print(Y)

X_train, X_test, y_train, y_test = train_test_split(X,Y, 
                                                    test_size=0.3, 
                                                    random_state=40)

print("train shape: " +str(X_train.shape)+"  test shape: "+ str(X_test.shape))




clf=RandomForestClassifier(n_estimators=100)
scores = cross_val_score(clf, X_train, y_train, cv=4,scoring='f1_macro')
print("f1-score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


rf = clf.fit(X_train, y_train) 
y_predicted = rf.predict(X_test)

print("F1 Random Forest on test set:", f1_score(y_test, y_predicted,average='macro'))

[    1     3     4   341   342   343   344   345   346   347   348   349
   350   351   352   354   355   356   357   360   361   362   363   364
   365   366   367   368   369   370   371   372   373   374   375   376
   377   378   379   381   382   383   384   387   388   389   390   391
   392   394   395   396   397   407   412   413   415   416   417   418
   419   420   421   422   423   424   425   426   427   428   429   430
   431   432   433   434   435   436   437   438   439   440   441   442
   443   444   445   446   447   448   449   450   451   452   453  2621
  2622  2623  2624  2626  2627  2628  2629  2631  2632  2637  2640  2642
  2644  2646  2649  2650  2652  2653  2655  2656  2659  2662  2666  2668
  2671  2673  2679  2682  2684  2686  2689  2690  2692  2693  2694  2696
  2699  2701  2702  2703  2704  2707  2711  2712  2776  3091  3324  3325
  3326  3327  3328  3329  3330  3331  3332  3333  3334  3335  3336  3337
  3338  3339  3340  3341  3342  3343  3344  3345  3

In [27]:
df['Region'][df.index.isin(ind_empty)]=lb.inverse_transform(rf.predict(filter_df[filter_df.index.isin(ind_empty)]))
df['Region'].unique()

/home/david/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


array(['ETELAE-SUOMI', 'POHJOIS-SUOMI', 'LAENSI-SUOMI', 'ITAE-SUOMI',
       'PELOPONNISOS', 'KENTRIKIMAKEDONIA', 'KRITI', 'VOREIOAIGAIO',
       'ANATOLIKIMAKEDONIA,THRAKI', 'THESSALIA', 'IPEIROS', 'IONIANISIA',
       'DYTIKIELLADA', 'STEREAELLADA', 'ATTIKI', 'NOTIOAIGAIO', 'ACORES',
       'NORTE', 'CENTRO', 'LISBOAEVALEDOTEJO', 'ALGARVE', 'MADEIRA',
       'SLOVENSKAREPUBLIKA', 'ROMACRO2SE', 'SWITZERLAND', 'DALMATIA',
       'KVARNER', 'ISTRIA', 'OBEROESTERREICH', 'NIEDEROESTERREICH',
       'KAERNTEN', 'STEIERMARK', 'SALZBURG', 'TIROL', 'LOMBARDIA',
       'PROV.AUT.DITRENTO', 'EMILIAROMAGNA', 'TOSCANA',
       'FRIULIVENEZIAGIULIA', 'PUGLIA', 'CAMPANIA', 'SICILIA', 'SARDEGNA',
       'PIEMONTE', 'LIGURIA', 'VENETO', 'MARCHE', 'LAZIO', 'ABRUZZO',
       'MOLISE', 'BASILICATA', 'CALABRIA', 'BRANDENBURG', 'EST',
       'KOEZEP-DUNANTUL', 'SLOVENIJA', 'ESTONIA', 'BADEN-WUERTTEMBERG',
       'NORDRHEIN-WESTFALEN', 'NIEDERSACHSEN', 'SACHSEN',
       'SACHSEN-ANHALT', 'THUERINGEN', 'BAY

# PREDICT PROVINCE AND COMMUNE

#### THERE ARE TOO MANY VALUES WITH JUST FEW FREQUENCY, SO I DECIDED TO DROP THESE VARIABLES

In [28]:
df['Province']=df['Province'].apply(remove)
print("#Province: "+str(len(df.Province.unique())))
print("Province: "+str(df.Province.unique()))

#Province: 944
Province: ['UUSIMAA' nan 'ITAE-UUSIMAA' 'PAEIJAET-HAEME' 'VARSINAIS-SUOMI'
 'SATAKUNTA' 'KANTA-HAEME' 'PIRKANMAA' 'KYMENLAAKSO' 'ETELAE-KARJALA'
 'ETELAE-SAVO' 'POHJOIS-SAVO' 'POHJOIS-KARJALA' 'KAINUU' 'KESKI-SUOMI'
 'ETELAE-POHJANMAA' 'POHJANMAA' 'KESKI-POHJANMAA' 'POHJOIS-POHJANMAA'
 'LAPPI' 'EVROS' 'XANTHI' 'RODOPI' 'KAVALA' 'THESSALONIKI' 'PIERIA'
 'SERESS' 'CHALKIDIKI' 'LARISA' 'MAGNISIA' 'ARTA' 'THESPROTIA' 'PREVEZA'
 'ZAKYNTHOS' 'KERKYRA' 'KEFALLINIA' 'LEFKADA' 'AITOLOAKARNANIA' 'ACHAIA'
 'ILEIA' 'VOIOTIA' 'EVVOIA' 'FTHIOTIDA' 'FOKIDA' 'ARGOLIDA' 'ARKADIA'
 'KORINTHIA' 'LAKONIA' 'MESSINIA' 'ATTIKI' 'LESVOS' 'SAMOS' 'CHIOS'
 'DODEKANISOS' 'DODEKASISOS' 'DODEKANOSOS' 'KYKLADES' 'IRAKLEIO' 'LASITHI'
 'RETHYMNI' 'CHANIA' 'DAO-LAFOES' 'PINHALINTERIORNORTE' 'GUARDA'
 'GRANDELISBOA' 'LISBOA' 'BAIXOMONDEGO' 'BRAGANCA' 'COIMBRA' 'BRAGA'
 'CASTELOBRANCO' 'SETUBA' 'COVADABEIRA' 'ALGARVE' 'AVEIRO' 'LEIRIA'
 'BEIRAINTERIORSUL' 'MADEIRA' 'VIANADOCASTELO' 'PORTO' 'BRAGAN?A' 'VIS

 'VLORE']


In [29]:
vc=df.Province.value_counts()

vals_to_remove = vc[vc <= 3].index.values

print(len(vals_to_remove))
len(vc)
#df['Province'].loc[df['Province'].isin(vals_to_remove)] = ""
#df.Province



321


943

In [30]:
df=df.drop(['Commune','Province'],axis=1)
df=df.reset_index(drop=True)

# ABOUT "YEAR" VARIABLES

#### JUST RENAMED THESE VARIABLE

In [31]:


def replaced(row):
    l1=['CG','B','NF','NC','CI','NS']
    l2=['comp_mand&guide','closed','ins_sampled','nocomp_mand','comp_mand','no_sampled']
    for l in range(0,len(l1)):
        if(row==l1[l]):
            row=l2[l]
            break
        else:
            row=row
    return row

for date in ['y1990','y1991','y1992','y1993','y1994','y1995','y1996','y1997','y1998','y1999','y2000','y2001','y2002','y2003','y2004','y2005','y2006','y2007','y2008','y2009','y2010','y2011','y2012','y2013','y2014','y2015','y2016','y2017']:
    df[date]=df[date].apply(replaced)



In [32]:
df.y2016.dropna().value_counts() #EXAMPLE

excellent                 17638
good                       1687
sufficient                  482
new                         302
poor                        300
insufficiently sampled      103
closed                       66
changes                      38
Name: y2016, dtype: int64

# MICROANALYSIS 

## ITALY

#### STUDY OF QUALITY IN ITALY FOR REGION 

In [33]:
df.Region[df.cc=='IT'].value_counts()

SICILIA                837
PUGLIA                 684
SARDEGNA               662
CALABRIA               651
LAZIO                  411
LIGURIA                410
TOSCANA                375
CAMPANIA               351
LOMBARDIA              275
MARCHE                 247
VENETO                 169
EMILIAROMAGNA          124
ABRUZZO                123
PIEMONTE                95
FRIULIVENEZIAGIULIA     67
BASILICATA              60
PROV.AUT.DITRENTO       39
MOLISE                  33
Name: Region, dtype: int64

In [34]:
REGIONI=df.Region[df.cc=='IT'].value_counts().index
Reg_Italy={}


df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
for anno in ['y2013','y2014','y2015','y2016','y2017']:
    for reg in REGIONI:
        SOMMA=0
        nex=0
        ngo=0
        nsu=0
        npo=0
        a=df[df.Region==reg][anno]

        for ai in a:
            if(ai=='excellent'):
                SOMMA+=1/excellent
                nex+=1


            elif (ai=='good'):
                SOMMA+=0
                ngo+=1
            elif (ai=='sufficient'):
                SOMMA+=-1/sufficient
                nsu+=1
            elif (ai=='poor'):
                SOMMA+=-1/poor
                npo+=1

        media=(((nex*1/excellent)+(nsu*(-1/sufficient))+(npo*(-1/poor)))/(npo+ngo+nex+nsu))*10**5
        Reg_Italy[reg]=100*np.exp(media)/5
        if (Reg_Italy[reg])>100: 
                    Reg_Italy[reg]=100
        elif(Reg_Italy[reg]<0):
                    Reg_Italy[reg]=0
    df_italy[anno]=Reg_Italy.values()
    
df_italy       


#we use the new metric for calculate the level of the quality for each region. 



,y2013,y2014,y2015,y2016,y2017
SICILIA,28.666264,24.115880,17.170481,15.816625,12.739043
PUGLIA,51.119483,46.950939,51.966225,54.220567,49.820040
SARDEGNA,41.624593,51.266550,45.208621,44.523561,39.458489
CALABRIA,1.436613,3.779288,6.662737,5.262986,3.516219
LAZIO,11.351107,21.429602,25.867538,19.521801,24.467081
LIGURIA,2.692098,12.270345,16.639403,17.300970,35.930008
TOSCANA,24.807079,17.635412,15.656641,19.109400,44.420185
CAMPANIA,0.008657,0.110906,0.195956,0.280381,0.498581
LOMBARDIA,23.894683,30.680244,16.433849,9.333705,4.712139
MARCHE,0.859796,0.612178,0.505925,0.595345,1.098255


In [35]:
south_italy=[0,1,2,3,7,11,15,17]
center_italy=[4,6,9]
north_italy=[5,8,10,12,13,14,16]

df_italy_min=pd.DataFrame(index=['south','center','north'],columns=['y2013','y2014','y2015','y2016','y2017'])
df_italy_min.loc['south']=df_italy.iloc[south_italy,:].mean(axis=0).values
df_italy_min.loc['center']=df_italy.iloc[center_italy,:].mean(axis=0).values
df_italy_min.loc['north']=df_italy.iloc[north_italy,:].mean(axis=0).values
df_italy_min
#we can't plot each region, so I decided divide for sud, north and center italy

,y2013,y2014,y2015,y2016,y2017
south,37.137,30.0315,20.7518,23.7446,21.1319
center,12.3393,13.2257,14.01,13.0755,23.3285
north,22.2294,24.114,25.1354,30.1927,32.1015


In [64]:


# Create a blank figure with labels
p = figure(plot_width = 800, plot_height = 600, 
           title = 'WATER QUALITY IN ITALY',
           x_axis_label = 'Year', y_axis_label = 'Quality')

# Example data
x = ['2013','2014','2015','2016','2017']
y1 = df_italy_min.iloc[0,:].values
y2= df_italy_min.iloc[1,:].values
y3 = df_italy_min.iloc[2,:].values
#circles_x = [9, 12, 4, 3, 15]
#circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
#p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(x, y1, size = 13, color = 'navy')
p.circle(x, y2, size = 13, color = 'red')
p.circle(x, y3, size = 13, color = 'green')

p.line(x, y1, line_width=5, color = 'navy', alpha = 0.6,legend='SOUTH')
p.line(x, y2, line_width=5, color = 'red', alpha = 0.6,legend='CENTER')
p.line(x, y3, line_width=5, color = 'green', alpha = 0.6,legend='NORTH')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

Loading BokehJS ...

In [37]:
df_print=pd.DataFrame(df_italy)
df_print['mean_quality']=df_italy.mean(axis=1)
df_print.sort_values(by='mean_quality')


#for last, calculate the mean of quality for each region and sort the results





,y2013,y2014,y2015,y2016,y2017,mean_quality
ABRUZZO,0.000020,0.000001,0.000058,0.006387,0.050064,0.011306
CAMPANIA,0.008657,0.110906,0.195956,0.280381,0.498581,0.218896
MARCHE,0.859796,0.612178,0.505925,0.595345,1.098255,0.734300
CALABRIA,1.436613,3.779288,6.662737,5.262986,3.516219,4.131569
PIEMONTE,4.180163,2.309574,6.261169,6.261169,3.822822,4.566979
LIGURIA,2.692098,12.270345,16.639403,17.300970,35.930008,16.966565
LOMBARDIA,23.894683,30.680244,16.433849,9.333705,4.712139,17.010924
SICILIA,28.666264,24.115880,17.170481,15.816625,12.739043,19.701659
LAZIO,11.351107,21.429602,25.867538,19.521801,24.467081,20.527426
MOLISE,55.575026,53.713652,1.335512,4.680835,0.116382,23.084282


In [38]:
df_italy

,y2013,y2014,y2015,y2016,y2017,mean_quality
SICILIA,28.666264,24.115880,17.170481,15.816625,12.739043,19.701659
PUGLIA,51.119483,46.950939,51.966225,54.220567,49.820040,50.815451
SARDEGNA,41.624593,51.266550,45.208621,44.523561,39.458489,44.416363
CALABRIA,1.436613,3.779288,6.662737,5.262986,3.516219,4.131569
LAZIO,11.351107,21.429602,25.867538,19.521801,24.467081,20.527426
LIGURIA,2.692098,12.270345,16.639403,17.300970,35.930008,16.966565
TOSCANA,24.807079,17.635412,15.656641,19.109400,44.420185,24.325743
CAMPANIA,0.008657,0.110906,0.195956,0.280381,0.498581,0.218896
LOMBARDIA,23.894683,30.680244,16.433849,9.333705,4.712139,17.010924
MARCHE,0.859796,0.612178,0.505925,0.595345,1.098255,0.734300


# EUROPE

#### NOW CHECK THE DIFFERENT BEETWEEN ITALY AND OTHERS EUROPEAN COUNTRIES OF THE MEDITERRANEAN AREA 

In [39]:
df_med=df[df['SeaRegion']=='Mediterranean Sea']
df_med

,cc,SeaRegion,Region,BWName,BWaterCat,Longitude_BW,Latitude_BW,y1990,y1991,y1992,...,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017,y2017 comment
331,GR,Mediterranean Sea,PELOPONNISOS,PANTAZI,Coast,22.2936,36.8170,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,new,excellent,excellent,excellent,"New BW, BWType = 2"
332,GR,Mediterranean Sea,PELOPONNISOS,VOIDOKILIA,Coast,21.6620,36.9634,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
333,GR,Mediterranean Sea,PELOPONNISOS,VROMONERI,Coast,21.6205,37.0181,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
334,GR,Mediterranean Sea,PELOPONNISOS,MATI,Coast,21.6462,36.9959,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
335,GR,Mediterranean Sea,PELOPONNISOS,XECHORIATIS,Coast,20.7948,37.8446,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
336,GR,Mediterranean Sea,PELOPONNISOS,LIMNIONAS,Coast,20.7012,37.7400,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
337,GR,Mediterranean Sea,PELOPONNISOS,PORTO ROXA,Coast,20.7101,37.7285,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
338,GR,Mediterranean Sea,PELOPONNISOS,XIGKIA,Coast,20.7351,37.8678,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
339,GR,Mediterranean Sea,PELOPONNISOS,MAKRIS GIALOS,Coast,20.7259,37.8782,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."
340,GR,Mediterranean Sea,PELOPONNISOS,AG.NIKOLAOS,Coast,20.7059,37.9075,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,new,"Assessed with data from 2017 onward. New BW,..."


In [73]:
stati=df_med.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe
REGIONI=df.Region[df.cc=='IT'].value_counts().index
Reg={}
import numpy as np


def metric(REGIONI,stati,df,df_europe):
    df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
    for state in stati:
        REGIONI=df.Region[df.cc==state].value_counts().index
        Reg={}
        df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])

        for anno in ['y2013','y2014','y2015','y2016','y2017']:
            for reg in REGIONI:
                SOMMA=0
                nex=0
                nsu=0
                ngo=0
                npo=0
                a=df[df.Region==reg][anno]

                for ai in a:
                    if(ai=='excellent'):
                        SOMMA+=1/excellent
                        nex+=1


                    elif (ai=='good'):
                        SOMMA+=0
                        ngo+=1
                    elif (ai=='sufficient'):
                        SOMMA+=-1/sufficient
                        nsu+=1
                    elif (ai=='poor'):
                        SOMMA+=-1/poor
                        npo+=1


                try:
                    media=(((nex*1/excellent)+(nsu*(-1/sufficient))+(npo*(-1/poor)))/(npo+ngo+nex+nsu))*10**5
                except:
                    media=np.nan
                Reg[reg]=100*np.exp(media)/5
                if (Reg[reg])>100: 
                    Reg[reg]=100
                elif(Reg[reg]<0):
                    Reg[reg]=0

            try:
                df_italy[anno]=Reg.values()
            except:pass
            df_europe.loc[state]=df_italy.mean(axis=0).values
            
    return df_europe
    
    
df_europe=metric(REGIONI,stati,df,df_europe)

#calculate metric of every region for every state
print(df_europe)

      y2013     y2014        y2015        y2016       y2017
GR   44.767   59.9012      61.0567      59.0865     58.9113
CH  57.4407   6.47788      9.90544      11.9772     11.8631
HR  39.1819   42.4981      53.3818      55.8868     59.2033
IT  27.2067   24.9293      21.3329       24.474      25.764
SI  46.1965    8.1308      8.32763      1.65191     20.0888
FR  1.78285   1.81113      2.06754      2.21603     2.92824
CY  61.5552   61.5552      61.5552      61.5552     60.9349
BG      NaN  0.866027     0.422446       1.3063    0.697885
ES  16.8857   17.2393      17.9552      19.4993     19.8345
MT  60.7649   61.5552      59.9848      60.7649     60.7649
AL      NaN       NaN  2.90156e-10  0.000158811  0.00128314


In [74]:
df_europe.index=['GREECE','SWITZERLAND','CROATIA','ITALY','SLOVENIA','FRANCE','CYPRUS','BULGARIA','SPAIN','MALTA','ALBANIA']
df_europe2=df_europe
df_europe2['mean_quality']=df_europe2.mean(axis=1)
df_europe2.sort_values(by='mean_quality')


,y2013,y2014,y2015,y2016,y2017,mean_quality
ALBANIA,NaN,NaN,2.90156e-10,0.000158811,0.00128314,0.000481
BULGARIA,NaN,0.866027,0.422446,1.3063,0.697885,0.823165
FRANCE,1.78285,1.81113,2.06754,2.21603,2.92824,2.161159
SLOVENIA,46.1965,8.1308,8.32763,1.65191,20.0888,16.879116
SPAIN,16.8857,17.2393,17.9552,19.4993,19.8345,18.282826
SWITZERLAND,57.4407,6.47788,9.90544,11.9772,11.8631,19.532866
ITALY,27.2067,24.9293,21.3329,24.474,25.764,24.741360
CROATIA,39.1819,42.4981,53.3818,55.8868,59.2033,50.030366
GREECE,44.767,59.9012,61.0567,59.0865,58.9113,56.744536
MALTA,60.7649,61.5552,59.9848,60.7649,60.7649,60.766924


In [77]:
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum


K={}
j=0
for k in df_europe2.index:
    K[k]=df_europe2['mean_quality'][j]
    j+=1
K

x = K

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="QUALITY OF WATER IN EUROPE", toolbar_location=None,
        tools="hover", tooltips="@country: @value")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

show(p)

In [48]:
df_europe


,y2013,y2014,y2015,y2016,y2017,mean_quality
GREECE,44.767,59.9012,61.0567,59.0865,58.9113,56.744536
SWITZERLAND,57.4407,6.47788,9.90544,11.9772,11.8631,19.532866
CROATIA,39.1819,42.4981,53.3818,55.8868,59.2033,50.030366
ITALY,27.2067,24.9293,21.3329,24.474,25.764,24.741360
SLOVENIA,46.1965,8.1308,8.32763,1.65191,20.0888,16.879116
FRANCE,1.78285,1.81113,2.06754,2.21603,2.92824,2.161159
CYPRUS,61.5552,61.5552,61.5552,61.5552,60.9349,61.431129
BULGARIA,NaN,0.866027,0.422446,1.3063,0.697885,0.823165
SPAIN,16.8857,17.2393,17.9552,19.4993,19.8345,18.282826
MALTA,60.7649,61.5552,59.9848,60.7649,60.7649,60.766924


# EUROPE'S RIVERS, LAKE, COAST AND TRANSITIONAL

#### CHECK THE QUALITY OF EACH RIVER, COAST, LAKE AND TRANSITIONAL

## RIVER 

In [78]:
df_med=df[df['SeaRegion']=='Mediterranean Sea']

df_med=df_med[df_med.BWaterCat=='River']


stati=df_med.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe
REGIONI=df.Region.value_counts().index
Reg_Italy={}
import numpy as np

df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe=metric(REGIONI,stati,df_med,df_europe)


    
    




In [79]:
df_europe=df_europe.drop('CH',axis=0)
df_europe.index=['ITALY','SLOVENIA','FRANCE','SPAIN']
df_europe2=df_europe
df_europe2['mean_quality']=df_europe2.mean(axis=1)
df_europe2.sort_values(by='mean_quality')
df_europe2

,y2013,y2014,y2015,y2016,y2017,mean_quality
ITALY,6.66697,6.66697,6.6732,23.0536,16.3869,11.889518
SLOVENIA,23.4843,0.0876026,0.00038371,0.00038371,32.3797,11.190485
FRANCE,0.0014865,8.77999,8.77348,0.00588414,0.129944,3.538157
SPAIN,24.7745,32.6221,25.3902,26.5312,24.3511,26.733828


In [80]:
K={}
j=0
for k in df_europe2.index:
    K[k]=df_europe2['mean_quality'][j]
    j+=1
K

{'ITALY': 11.889518256567538,
 'SLOVENIA': 11.190484835036909,
 'FRANCE': 3.5381566243379106,
 'SPAIN': 26.733828036029955}

In [82]:
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum

x = K

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="RIVER", toolbar_location=None,
        tools="hover", tooltips="@country: @value")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

show(p)

In [53]:
REGIONI

Index(['MEDITERRANEE', 'OUEST', 'SICILIA', 'PUGLIA', 'SARDEGNA', 'CALABRIA',
       'GALICIA', 'DALMATIA', 'SWITZERLAND', 'LAZIO',
       ...
       'TIROL', 'NYUGAT-DUNANTUL', 'STEIERMARK', 'NORTHWEST(UK)', 'SACHSEN',
       'KAERNTEN', 'NORTHEAST(UK)', 'POHJOIS-SUOMI', 'MOLISE',
       'NIEDEROESTERREICH'],
      dtype='object', length=118)

## LAKE

In [83]:
df_med=df[df['SeaRegion']=='Mediterranean Sea']

df_med=df_med[df_med.BWaterCat=='Lake']


stati=df_med.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe
REGIONI=df.Region.value_counts().index
Reg_Italy={}
import numpy as np

df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe=metric(REGIONI,stati,df_med,df_europe)


df_europe.index=['GREECE','SWITZERLAND','ITALY','FRANCE','SPAIN']
df_europe2=df_europe
df_europe2['mean_quality']=df_europe2.mean(axis=1)
df_europe2.sort_values(by='mean_quality')
print(df_europe2)


K={}
j=0
for k in df_europe2.index:
    K[k]=df_europe2['mean_quality'][j]
    j+=1

x = K

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="LAKE", toolbar_location=None,
        tools="hover", tooltips="@country: @value")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

show(p)


               y2013    y2014    y2015    y2016    y2017  mean_quality
GREECE       35.0871  61.5552  61.5552       20  61.5552     47.950529
SWITZERLAND  58.0779  13.6954   22.379  17.4765  5.84731     23.495236
ITALY        44.2247  43.4043  42.6003  47.5723  46.9009     44.940522
FRANCE       43.9304  44.8967   45.717  49.8639  44.4402     45.769641
SPAIN        55.0655  55.0655  49.5609  41.2498  52.0481     50.597970


## COAST

In [84]:
df_med=df[df['SeaRegion']=='Mediterranean Sea']

df_med=df_med[df_med.BWaterCat=='Coast']


stati=df_med.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe
REGIONI=df.Region.value_counts().index
Reg_Italy={}
import numpy as np

df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe=metric(REGIONI,stati,df_med,df_europe)

    

df_europe.index=['GREECE','CROATIA','ITALY','SLOVENIA','FRANCE','CYPRUS','BULGARIA','SPAIN','MALTA','ALBANIA']
df_europe2=df_europe
df_europe2['mean_quality']=df_europe2.mean(axis=1)
df_europe2.sort_values(by='mean_quality')
df_europe2

K={}
j=0
for k in df_europe2.index:
    K[k]=df_europe2['mean_quality'][j]
    j+=1

x = K

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="COAST", toolbar_location=None,
        tools="hover", tooltips="@country: @value")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

show(p)

## TRANSITIONAL

In [58]:
df_med=df[df['SeaRegion']=='Mediterranean Sea']

df_med=df_med[df_med.BWaterCat=='Transitional']


stati=df_med.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe
REGIONI=df.Region.value_counts().index
Reg_Italy={}
import numpy as np

df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe=metric(REGIONI,stati,df_med,df_europe)

    
df_europe=df_europe.drop('FR',axis=0)
df_europe.index=['CROATIA','ITALY','SPAIN']
df_europe2=df_europe
df_europe2['mean_quality']=df_europe2.mean(axis=1)
df_europe2.sort_values(by='mean_quality')
df_europe2

K={}
j=0
for k in df_europe2.index:
    K[k]=df_europe2['mean_quality'][j]
    j+=1

x = K

data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(plot_height=350, title="Pie Chart", toolbar_location=None,
        tools="hover", tooltips="@country: @value")

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)

show(p)


# ITALY VS EUROPE

In [100]:
df_med=df[df['SeaRegion']=='Mediterranean Sea']

europe=df_med[df_med.cc!='IT']
italy=df_med[df_med.cc=='IT']


In [101]:
df2=pd.DataFrame(index=['EUROPE','ITALY'],columns=['y2013','y2014','y2015','y2016','y2017'])


stati=europe.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe
REGIONI=df.Region.value_counts().index
Reg_Italy={}

df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe=metric(REGIONI,stati,europe,df_europe)
df2.loc['EUROPE']=df_europe.mean(axis=0)


stati=italy.cc.unique()
df_europe=pd.DataFrame(index=stati,columns=['y2013','y2014','y2015','y2016','y2017'])
REGIONI=df.Region.value_counts().index
Reg_Italy={}

df_italy=pd.DataFrame(index=REGIONI,columns=['y2013','y2014','y2015','y2016','y2017'])
df_europe=metric(REGIONI,stati,italy,df_europe)
df2.loc['ITALY']=df_europe.mean(axis=0)


df2

,y2013,y2014,y2015,y2016,y2017
EUROPE,43.5678,38.3797,31.5068,30.0004,34.4616
ITALY,27.2074,24.9296,21.3327,24.5167,25.7603


In [104]:
df2.loc['ITALY'].values

array([27.20738577636191, 24.929565186956797, 21.33272394410961,
       24.516663228703845, 25.760256184335336], dtype=object)

In [108]:


# Create a blank figure with labels
p = figure(plot_width = 800, plot_height = 600, 
           title = 'EUROPE VS ITALY',
           x_axis_label = 'Year', y_axis_label = 'Quality')

# Example data
x = ['2013','2014','2015','2016','2017']
y1 = df2.loc['ITALY'].values
y2= df2.loc['EUROPE'].values
#circles_x = [9, 12, 4, 3, 15]
#circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
#p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(x, y1, size = 13, color = 'navy')
p.circle(x, y2, size = 13, color = 'red')

p.line(x, y1, line_width=5, color = 'navy', alpha = 0.6,legend='ITALY')
p.line(x, y2, line_width=5, color = 'red', alpha = 0.6,legend='EUROPE')






# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

Loading BokehJS ...

# NEXT STEP (I DIDN'T FINISH IT)

# TEXTMINING

In [127]:
dft=df.dropna()

from mltools.textMining import text_preprocessing

tp = text_preprocessing.TextPreprocessing(chr_to_remove=[""],
                 chr_to_keep=r"[^A-Za-z0123456789]",lemmatization = False, remove_stopwords=True)

training_clean = tp.fit(dft, 'y2017 comment', min_len=1)


Data cleaning...
Standardization...
Tokenization...
Removing stopwords...
Finish


In [124]:
training_clean

,cc,SeaRegion,Region,BWName,BWaterCat,Longitude_BW,Latitude_BW,y1990,y1991,y1992,...,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017,y2017 comment,tokens
5167,IT,Mediterranean Sea,LIGURIA,STAZIONE FF.SS.,Coast,7.6639,43.7777,comp_mand&guide,comp_mand&guide,nocomp_mand,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5168,IT,Mediterranean Sea,LIGURIA,PASSAGGIO A LIVELLO,Coast,7.6671,43.7771,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5169,IT,Mediterranean Sea,LIGURIA,EX CASINĂ’,Coast,7.6709,43.7766,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5172,IT,Mediterranean Sea,LIGURIA,LEVANTE PUNTA MIGLIARESE,Coast,7.6895,43.7878,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5173,IT,Mediterranean Sea,LIGURIA,MADONNA DELLA RUOTA,Coast,7.6943,43.7935,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5174,IT,Mediterranean Sea,LIGURIA,GIUNCHETTO,Coast,7.6975,43.7972,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5190,IT,Mediterranean Sea,LIGURIA,CROCE ROSSA,Coast,8.0793,43.9055,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,ins_sampled,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5191,IT,Mediterranean Sea,LIGURIA,VIA CAIROLI,Coast,8.0820,43.9076,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,ins_sampled,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5213,IT,Mediterranean Sea,LIGURIA,VILLA SADA,Coast,7.7309,43.7965,comp_mand&guide,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"
5240,IT,Mediterranean Sea,LIGURIA,CORSO MAZZINI,Coast,7.8048,43.8231,nocomp_mand,comp_mand&guide,comp_mand&guide,...,comp_mand&guide,comp_mand&guide,comp_mand&guide,excellent,excellent,excellent,excellent,excellent,group status calculated,"[group, status, calculated]"


# CLOSED

In [ ]:
df_med=df[df.SeaRegion=='Mediterranean Sea']
df_med

In [ ]:


print('2009: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2009=='closed'])))
print('2010: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2010=='closed'])))
print('2011: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2011=='closed'])))
print('2012: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2012=='closed'])))
print('2013: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2013=='closed'])))
print('2014: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2014=='closed'])))
print('2015: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2015=='closed'])))
print('2016: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2016=='closed'])))
print('2017: '+ str(len(df_med[['y2009','y2010','y2011']][df_med.y2017=='closed'])))


In [ ]:
df2=df_med[df_med.cc=='IT']
#df2=df_med
def f2(lista,df_med):
    k=df_med[lista][df_med[lista[0]]=='comp_mand']
    print(lista[0][1:]+": "+str(len(k)))
    for anno in lista[1:]:
        k=k[lista][k[anno]=='comp_mand']
        print(anno[1:]+": "+str(len(k)))





lista=['y2009','y2010','y2011','y2012','y2013','y2014','y2015','y2016','y2017']
for i in range(0,len(lista)):
    print(lista[i])
    l=lista[i:]
    f2(l,df2)
    print("----------")

In [ ]:
df_med.y2009.unique()

In [ ]:
kt='comp_mand&guide'


def pri(dh,kt):
    print('2009: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2009==kt])*100/len(dh)))
    print('2010: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2010==kt])*100/len(dh)))
    print('2011: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2011==kt])*100/len(dh)))
    print('2012: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2012==kt])*100/len(dh)))
    print('2013: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2013==kt])*100/len(dh)))
    print('2014: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2014==kt])*100/len(dh)))
    print('2015: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2015==kt])*100/len(dh)))
    print('2016: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2016==kt])*100/len(dh)))
    print('2017: '+ str(len(dh[['y2009','y2010','y2011']][dh.y2017==kt])*100/len(dh)))


print("NO ITA")
dh=df_med[df_med.cc!='FR']

pri(dh,kt)
print("ITA")
dh=df_med[df_med.cc=='FR']

pri(dh,kt)



In [ ]:
k=df_med[['y2009','y2010','y2011','y2012','y2013','y2014','y2015','y2016','y2017']][df_med['y2009']=='closed']
len(k[['y2009','y2010','y2011','y2012','y2013','y2014','y2015','y2016','y2017']][k['y2010']=='closed'])

# MAP

In [65]:


df2=df[df.cc=='IT']

df2=df2[['cc','Longitude_BW','Latitude_BW','y2015','y2016','y2017']].dropna()
df2=df2[df2.y2015.isin(['poor','sufficient','good','excellent'])]
df2=df2[df2.y2016.isin(['poor','sufficient','good','excellent'])]
df2=df2[df2.y2017.isin(['poor','sufficient','good','excellent'])]

#south=SICILIA PUGLIA SARDEGNA CALABRIA CAMPANIA ABBRUZZO BASILIACATA MOLISE
#center=LAZIO TOSCANA MARCHE
#NORT=LIGURSICILIA PUGLIA SARDEGNA CALABRIA CAMPANIA ABBRUZZO BASILIACATA MOLISEIA LOMBARDIA VENETO EMILIAROMAGNA PIEMONTE FRIULIVENEZIAGIULIA PROV.AUT.DITRENTO



IT_LONG=df2.Longitude_BW.reset_index(drop=True)
IT_LAT=df2.Latitude_BW.reset_index(drop=True)



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium

#df_traffic = pd.read_csv('../input/ukTrafficAADF.csv')

map_hooray = folium.Map(location=[41.89193, 12.51133],
                    zoom_start = 5) # Uses lat then lon. The bigger the zoom number, the closer in you get
map_hooray # Calls the map to display

for num in range(0,len(df2)):
    if df2.y2017.iloc[num]=='excellent':
        colore='blue'
    elif df2.y2017.iloc[num]=='good':
        colore='green'
    elif df2.y2017.iloc[num]=='sufficient':
        colore='orange' 
    elif df2.y2017.iloc[num]=='poor':
        colore='red'
    folium.Marker([IT_LAT[num], IT_LONG[num]],
                        icon=folium.Icon(color=colore)
                        ).add_to(map_hooray)
    



map_hooray

map_hooray.save(outfile='map_Italy.html')
#map_osm = folium.Map(location=[54.7, -4.36])
#This 2nd set of coordinates will drop you down right in the middle of the UK, which is actually the seas because it's between mainland and N.Ireland,

In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


df2=df[df.SeaRegion=='Mediterranean Sea']
df2=df2[['Longitude_BW','Latitude_BW','y2015','y2016','y2017']].dropna()
df2=df2[df2.y2015.isin(['poor','sufficient','good','excellent'])]
df2=df2[df2.y2016.isin(['poor','sufficient','good','excellent'])]
df2=df2[df2.y2017.isin(['poor','sufficient','good','excellent'])]

#south=SICILIA PUGLIA SARDEGNA CALABRIA CAMPANIA ABBRUZZO BASILIACATA MOLISE
#center=LAZIO TOSCANA MARCHE
#NORT=LIGURSICILIA PUGLIA SARDEGNA CALABRIA CAMPANIA ABBRUZZO BASILIACATA MOLISEIA LOMBARDIA VENETO EMILIAROMAGNA PIEMONTE FRIULIVENEZIAGIULIA PROV.AUT.DITRENTO



IT_LONG=df2.Longitude_BW.reset_index(drop=True)
IT_LAT=df2.Latitude_BW.reset_index(drop=True)



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium

#df_traffic = pd.read_csv('../input/ukTrafficAADF.csv')

map_hooray = folium.Map(location=[41.89193, 12.51133],
                    zoom_start = 5) # Uses lat then lon. The bigger the zoom number, the closer in you get
map_hooray # Calls the map to display

for num in range(0,len(df2)):
    if df2.y2017.iloc[num]=='excellent':
        colore='green'
    elif df2.y2017.iloc[num]=='good':
        colore='yellow'
    elif df2.y2017.iloc[num]=='sufficient':
        colore='orange' 
    elif df2.y2017.iloc[num]=='poor':
        colore='red'
    folium.Marker([IT_LAT[num], IT_LONG[num]],
                        icon=folium.Icon(color=colore)
                        ).add_to(map_hooray)
    



map_hooray

map_hooray.save(outfile='map_europe20.html')

#map_osm = folium.Map(location=[54.7, -4.36])
#This 2nd set of coordinates will drop you down right in the middle of the UK, which is actually the seas because it's between mainland and N.Ireland,

In [67]:
len(df[df.y2017=='poor'])

281

In [101]:
df

,cc,SeaRegion,Region,Province,Commune,BWName,BWaterCat,Longitude_BW,Latitude_BW,y1990,...,y2009,y2010,y2011,y2012,y2013,y2014,y2015,y2016,y2017,y2017 comment
1320,GR,Mediterranean Sea,STEREA ELLADA,EVVOIA,MESSAPION,DAFNI,Coast,23.494600,38.6289,NaN,...,NF,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
18311,FR,Northwest Atlantic Ocean,DOM,ANTILLES GUYANE,GOURBEYRE,BASSIN BLEU - PALMISTE ...,River,-61.666000,16.0184,NaN,...,CG,NF,NF,NF,excellent,excellent,good,good,good,NaN
8054,IT,Mediterranean Sea,PUGLIA,FOGGIA,VIESTE,1 CANALE - 30M. SX DELLA FOCE,Coast,16.176300,41.8701,NaN,...,CG,NF,CG,CG,excellent,excellent,excellent,excellent,excellent,NaN
14346,DK,North-east Atlantic Ocean,NORDJYLLAND,LAESOE,LAESOE,SYD FOR VESTEROE HAVN,Coast,10.921600,57.2941,NaN,...,CI,CG,excellent,good,excellent,excellent,excellent,excellent,excellent,NaN
1376,GR,Mediterranean Sea,STEREA ELLADA,FTHIOTIDA,MALESINIS,VIVOS,Coast,23.167800,38.6404,NaN,...,NF,CG,good,good,good,NaN,NaN,NaN,NaN,NaN
14522,DK,Baltic Sea,MIDTJYLLAND,SYDDJURS,SYDDJURS,LYNGSBAEK STRAND,Coast,10.605500,56.2276,NaN,...,CG,CG,excellent,excellent,excellent,excellent,good,good,good,NaN
18236,FR,Mediterranean Sea,MEDITERRANEE,CORSE,SANTA MARIA DI LOTA,MIOMO ...,Coast,9.461170,42.7405,NaN,...,CG,CG,CG,CI,excellent,excellent,excellent,excellent,excellent,NaN
202,FI,Baltic Sea,ITAE-SUOMI,POHJOIS-KARJALA,KONTIOLAHTI,ONKILAMPI,Lake,29.822000,62.6398,NaN,...,CG,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN
6139,IT,Mediterranean Sea,FRIULI VENEZIA GIULIA,TRIESTE,DUINO-AURISINA,SISTIANA - SOTTO IL CAMPING,Coast,13.623400,45.7686,CG,...,CG,CG,CI,CG,excellent,excellent,excellent,excellent,excellent,NaN
14262,DK,North-east Atlantic Ocean,NORDJYLLAND,MORSOE,MORSOE,SILLERSLEV SOMMERHUSOMRAADE,Coast,8.774990,56.6936,NaN,...,CG,CG,excellent,excellent,excellent,excellent,excellent,excellent,excellent,NaN


In [155]:
df2=df[df.cc=='IT']

In [189]:
df2=df[df.cc=='IT']
df2=df2[['Longitude_BW','Latitude_BW','y2015','y2016','y2017']].dropna()
df2=df2[df2.y2015.isin(['poor','sufficient','good','excellent'])]
df2=df2[df2.y2016.isin(['poor','sufficient','good','excellent'])]
df2=df2[df2.y2017.isin(['poor','sufficient','good','excellent'])].sample(2000)
df2
IT_LONG=df2.Longitude_BW.reset_index(drop=True)
IT_LAT=df2.Latitude_BW.reset_index(drop=True)


In [171]:
IT_LONG

0      15.9291
1      11.1082
2      18.3956
3       8.2557
4      10.3259
5       9.8569
6       8.5754
7      14.3806
8      17.2750
9      16.6278
10      8.7379
11     10.7190
12     15.1367
13     12.7989
14     15.8099
15     12.5959
16      8.5671
17     10.8128
18     10.2084
19      8.5099
20      8.7490
21      9.4557
22      8.7199
23      9.3134
24      8.5912
25     18.3842
26      9.0208
27     13.7515
28     13.4766
29      9.6537
        ...   
970    16.0683
971     9.6910
972     9.5237
973     8.1879
974    16.9152
975    12.9442
976    10.0102
977    13.9086
978    13.6787
979    15.1333
980    12.4981
981    16.7962
982     8.9323
983     9.3687
984    12.5005
985     8.5718
986     7.5595
987    13.7502
988     9.1246
989    10.2865
990    16.7707
991     7.8551
992    16.2174
993    15.6639
994    10.8810
995     9.9245
996    15.1085
997    13.6465
998    14.9291
999    14.4825
Name: Longitude_BW, Length: 1000, dtype: float64

In [175]:
df2.y2017.iloc[0]

'good'